In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import re
from PIL import Image
import pickle 
import random
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [16]:
def formatar(path, maxsize):
        img = Image.open(path).convert('L')   
        WIDTH, HEIGHT = img.size
        if WIDTH != HEIGHT:
                m_min_d = min(WIDTH, HEIGHT)
                img = img.crop((0, 0, m_min_d, m_min_d))
        img.thumbnail(maxsize, Image.LANCZOS)
        return np.asarray(img)

In [3]:
def carregar(path_dir, maxsize):
        images = []
        labels = []
        os.chdir(path_dir)
        for file in glob.glob("*.png"):
                img = formatar(file, maxsize)
                if re.match('Leve.*', file):
                        images.append(img)
                        labels.append(0)

                elif re.match('Carregado.*', file):
                        images.append(img)
                        labels.append(1)

                elif re.match('sobrecarregado.*', file):
                        images.append(img)
                        labels.append(2)
        return (np.asarray(images), np.asarray(labels))

In [17]:
maxsize = 250, 250
def display_images(images, labels):
        plt.figure(figsize=(80, 80))
        grid_size = min(55, len(images))
        for i in range(grid_size):
                plt.subplot(11, 5, i+1)
                plt.xticks([])
                plt.yticks([])
                plt.grid(False)
                plt.imshow(images[i], cmap=plt.cm.binary)
                plt.xlabel(class_names[labels[i]])

In [18]:
(train_images, train_labels) = carregar(r'C:\Users\gusta\OneDrive\Documentos\GitHub\CargoSecurity\treino', maxsize)
(test_images, test_labels) = carregar(r'C:\Users\gusta\OneDrive\Documentos\GitHub\CargoSecurity\teste', maxsize)

train_images = train_images / 255
test_images = test_images / 255

In [19]:
indices = list(range(len(train_labels)))

# Embaralhe os índices
np.random.shuffle(indices)

# Use os índices embaralhados para embaralhar as duas listas
train_images = train_images[indices]
train_labels = train_labels[indices]

class_names = [ 'leve', 'carregado', 'sobrecarregado']

In [20]:
model = Sequential() # 28x28
model.add(Conv2D(32, (3,3), activation='relu',input_shape=(250,250,1) )) #20x24x24
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) #40x8x8
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Conv2D(128, kernel_size=(3,3), activation='relu')) #40x8x8
model.add(MaxPooling2D(pool_size=(2,2))) 

model.add(Flatten())
model.add(Dense(192, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=150, batch_size=43)

In [11]:

def model_builder(hp):
  neuro1 = hp.Int('units_1', min_value=32, max_value=512, step=32)
  neuro2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
  neuro3 = hp.Int('units_3', min_value=32, max_value=512, step=32)
  neuro4 = hp.Int('units_4', min_value=32, max_value=512, step=32)
  dropout1 = hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)
  dropout2 = hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)
  
  model = Sequential() # 28x28
  model.add(Conv2D(units=neuro1,kernel_size=(3,3), activation='relu',input_shape=(250,250,1))) #20x24x24
  model.add(MaxPooling2D(pool_size=(2,2))) 
  model.add(Dropout(rate=dropout1))
  model.add(Conv2D(units=neuro2, kernel_size=(3,3), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) 
  model.add(Dropout(rate=dropout2))
  model.add(Conv2D(units=neuro3, kernel_size=(3,3), activation='relu')) #40x8x8
  model.add(MaxPooling2D(pool_size=(2,2))) 
  model.add(Flatten())
  model.add(Dense(units=neuro4, activation='relu'))
  model.add(Dense(3, activation='softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=1000,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_images, train_labels, epochs=1000, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
MELHORES PARAMETROS
Primeiro Neuronio: {best_hps.get('units_1')}
Segundo Neuronio: {best_hps.get('units_2')}
Terceiro Neuronio: {best_hps.get('units_3')}
Quarto Neuronio: {best_hps.get('units_4')}
Primeiro Dropout: {best_hps.get('dropout_1')}
Segundo Dropout: {best_hps.get('dropout_1')}
Taxa de aprendizagem: {best_hps.get('learning_rate')}.
""")

FailedPreconditionError: my_dir\intro_to_kt is not a directory

In [15]:
entrada = np.asarray([1,2,3,4])
saida = np.asarray([[2],[4],[6],[8]])

def model_builder(hp):
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  drop1 = hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)
  drop2 = hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)

  model = Sequential() # 28x28
  model.add(Dense(units=hp_units1,input_shape=[1]))
  model.add(Dropout(rate=drop1))
  model.add(Dense(units=hp_units, activation='relu'))
  model.add(Dropout(rate=drop2))
  model.add(Dense(1, activation='softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['accuracy'])

  return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(entrada, saida, epochs=2, validation_split=0.1, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Primeiro Neuronio {best_hps.get('units')} 
Segundo Neuronio {best_hps.get('units1')}
Primeiro drop {best_hps.get('dropout_1')} 
Segundo drop {best_hps.get('dropout_2')}
Taxa de aprendizagem {best_hps.get('learning_rate')}.
""")

Trial 128 Complete [00h 00m 04s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 06m 57s

Search: Running Trial #129

Value             |Best Value So Far |Hyperparameter
448               |448               |units
128               |256               |units1
0.4               |0.3               |dropout_1
0                 |0                 |dropout_2
0.0001            |0.001             |learning_rate
4                 |2                 |tuner/epochs
2                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
1                 |0                 |tuner/round
0030              |None              |tuner/trial_id

Epoch 3/4


KeyboardInterrupt: 

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
acerto = str(test_acc*100)
print(acerto)


1/1 [==============================] - 0s 157ms/step - loss: 11.3714 - accuracy: 0.4286
42.85714328289032


In [9]:
a = model.predict(test_images)
result = []
for x in a:
    x = x.tolist()
    result.append(x.index(max(x)))
print(result)
print(test_labels.tolist())

1/1 [==============================] - 0s 117ms/step
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 2, 2, 2]


In [24]:
from PIL import Image
import os

# Pasta de entrada contendo as imagens originais
pasta_de_entrada = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\teste'

# Pasta de saída onde as imagens redimensionadas serão salvas
pasta_de_saida = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\aa'

# Tamanho desejado das imagens (300x300)
novo_tamanho = (300, 300)

# Certifique-se de que a pasta de saída exista ou crie-a
if not os.path.exists(pasta_de_saida):
    os.makedirs(pasta_de_saida)

# Lista todos os arquivos na pasta de entrada
arquivos = os.listdir(pasta_de_entrada)

for arquivo in arquivos:
    try:
        # Abre a imagem original
        imagem = Image.open(os.path.join(pasta_de_entrada, arquivo))
        
        # Redimensiona a imagem para o tamanho desejado
        imagem_redimensionada = imagem.resize(novo_tamanho)
        
        # Salva a imagem redimensionada na pasta de saída
        imagem_redimensionada.save(os.path.join(pasta_de_saida, arquivo))
        
        print(f'{arquivo} redimensionado com sucesso!')
    except Exception as e:
        print(f'Erro ao redimensionar {arquivo}: {str(e)}')


Carregado (1).jpeg redimensionado com sucesso!
Carregado (1).jpg redimensionado com sucesso!
Carregado (1).png redimensionado com sucesso!
Leve (1).jpeg redimensionado com sucesso!
Leve (1).jpg redimensionado com sucesso!
Leve (1).png redimensionado com sucesso!
Leve (2).jpeg redimensionado com sucesso!
Leve (2).jpg redimensionado com sucesso!
Leve (2).png redimensionado com sucesso!
Leve (3).jpeg redimensionado com sucesso!
sobrecarregado005.png redimensionado com sucesso!
sobrecarregado006.png redimensionado com sucesso!
sobrecarregado007.png redimensionado com sucesso!


In [25]:
from PIL import Image
import os

pasta_de_entrada = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\teste'

# Pasta de saída onde as imagens redimensionadas serão salvas
pasta_de_saida = r'C:\Users\labin\OneDrive\Documentos\GitHub\Constru-o-da-Rede-Neural\aa'

# Certifique-se de que a pasta de saída exista ou crie-a
if not os.path.exists(pasta_de_saida):
    os.makedirs(pasta_de_saida)

# Lista todos os arquivos na pasta de entrada
arquivos = os.listdir(pasta_de_entrada)

for arquivo in arquivos:
    try:
        # Abre a imagem original
        imagem = Image.open(os.path.join(pasta_de_entrada, arquivo))
        
        # Define o formato de saída como PNG
        imagem.save(os.path.join(pasta_de_saida, f'{os.path.splitext(arquivo)[0]}.png'), 'PNG')
        
        print(f'{arquivo} convertido para PNG com sucesso!')
    except Exception as e:
        print(f'Erro ao converter {arquivo} para PNG: {str(e)}')


Carregado (1).jpeg convertido para PNG com sucesso!
Carregado (1).jpg convertido para PNG com sucesso!
Carregado (1).png convertido para PNG com sucesso!
Leve (1).jpeg convertido para PNG com sucesso!
Leve (1).jpg convertido para PNG com sucesso!
Leve (1).png convertido para PNG com sucesso!
Leve (2).jpeg convertido para PNG com sucesso!
Leve (2).jpg convertido para PNG com sucesso!
Leve (2).png convertido para PNG com sucesso!
Leve (3).jpeg convertido para PNG com sucesso!
sobrecarregado005.png convertido para PNG com sucesso!
sobrecarregado006.png convertido para PNG com sucesso!
sobrecarregado007.png convertido para PNG com sucesso!
